In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import statsmodels.api as sm
import os
import sys

sys.path.insert(0, os.path.abspath('../developer'))

from config import MOCK_DATA, CODE, OUT
from developer.utilities import read_yaml
from developer.analysis.model import load_model
import re

In [27]:
df_total = pd.read_csv(OUT / "data" / "data_regression.csv")

##### regression

In [28]:
# Select the design matrix (explanatory variables) and the outcome variables
outcome = 'utility'
explanatory_vars = [col for col in df_total.columns if "att" in col] + ['ID']


X = df_total[explanatory_vars].astype(int)
y = df_total[outcome].astype(int)

X = sm.add_constant(X)
model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups':X['ID']})

In [30]:
outcome = 'utility'
explanatory_vars = [col for col in df_total.columns if "att" in col] + ['ID']

df_control = df_total[df_total['treatment_status'] == 0]

X = df_control[explanatory_vars].astype(int)
y = df_control[outcome].astype(int)

X = sm.add_constant(X)
model_control = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups':X['ID']})

In [32]:
outcome = 'utility'
explanatory_vars = [col for col in df_total.columns if "att" in col] + ['ID']

df_treated = df_total[df_total['treatment_status'] == 1]

X = df_treated[explanatory_vars].astype(int)
y = df_treated[outcome].astype(int)

X = sm.add_constant(X)
model_treated = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups':X['ID']})

In [33]:
model_control.summary()

c:\ProgramData\Anaconda3\envs\conjoint\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning:

covariance of constraints does not have full rank. The number of constraints is 26, but rank is 4



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                utility   R-squared:                       0.216
Model:                            OLS   Adj. R-squared:                 -0.217
Method:                 Least Squares   F-statistic:                  0.009751
Date:                Tue, 08 Aug 2023   Prob (F-statistic):               1.00
Time:                        12:59:08   Log-Likelihood:                -65.662
No. Observations:                  60   AIC:                             175.3
Df Residuals:                      38   BIC:                             221.4
Df Model:                          21                                         
Covariance Type:              cluster                                         
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                     -0.1532      0.064     -2.399      0.016      -0.278      -0.028
att_1_PhaseOut                             0.3255      0.150      2.167      0.030       0.031       0.620
att_1_StatusQuo                           -0.0870      0.143     -0.610      0.542      -0.367       0.193
att_1_Stop&Maintain                       -0.3070      0.291     -1.056      0.291      -0.877       0.263
att_1_Stop&Reduce                         -0.0847      0.361     -0.235      0.814      -0.793       0.623
att_2_HighInvestment&Int                  -0.1978      0.318     -0.621      0.534      -0.822       0.426
att_2_HighInvestment&Int&Consideration    -0.2629      0.213     -1.236      0.217      -0.680       0.154
att_2_LowInvestment                        0.2660      0.306      0.869      0.385      -0.334       0.866
att_2_LowInvestment&LowConsideration       0.0415      0.115      0.360      0.719      -0.184       0.267
att_3_EnergyAccess                        -0.1364      0.163     -0.838      0.402      -0.455       0.183
att_3_HealthEdu                           -0.1922      0.217     -0.887      0.375      -0.617       0.233
att_3_LowPrices                            0.2624      0.220      1.192      0.233      -0.169       0.694
att_3_NothingSoc                           0.0416      0.451      0.092      0.926      -0.842       0.925
att_3_Transfers                           -0.1286      0.225     -0.572      0.567      -0.569       0.312
att_4_CreateJobs                           0.0002      0.332      0.000      1.000      -0.650       0.650
att_4_EarlyPension                         0.2609      0.219      1.191      0.234      -0.168       0.690
att_4_JobGuarantee                         0.0944      0.447      0.211      0.833      -0.782       0.971
att_4_NothingEco                          -0.0429      0.394     -0.109      0.913      -0.815       0.730
att_4_Retrain                             -0.4658      0.186     -2.505      0.012      -0.830      -0.101
att_5_CentralGov                           0.2764      0.370      0.748      0.455      -0.448       1.001
att_5_CivilNGO                             0.1582      0.658      0.240      0.810      -1.132       1.449
att_5_EnergySector                         0.0776      0.685      0.113      0.910      -1.264       1.420
att_5_LabourUnion                          0.2136      0.477      0.448      0.654      -0.721       1.148
att_5_LocalGov                            -0.1683      0.220     -0.764      0.445      -0.600       0.264
att_5_Media                               -0.3500      0.295     -1.186      0.236      -0.928       0.228
att_5_Researchers                         -0.3607      0.299     -1.208      0.227     

In [24]:
model = load_model(OUT / "models" / "model.pickle")

In [25]:
print("Regression results for Attribute A:")
print(model.summary())

Regression results for Attribute A:
                            OLS Regression Results                            
Dep. Variable:                utility   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                    0.5149
Date:                Tue, 08 Aug 2023   Prob (F-statistic):              0.845
Time:                        11:31:15   Log-Likelihood:                -137.13
No. Observations:                 132   AIC:                             318.3
Df Residuals:                     110   BIC:                             381.7
Df Model:                          21                                         
Covariance Type:              cluster                                         
                                             coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------

c:\ProgramData\Anaconda3\envs\conjoint\Lib\site-packages\statsmodels\base\model.py:1888: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 26, but rank is 10
  warnings.warn('covariance of constraints does not have full '


## Plots

##### 1.1 Relative support plot (Intra)

In [26]:
import plotly.graph_objects as go

# Assuming 'model' is the variable that holds your regression results
# Extract the coefficients and standard errors for each attribute level
att_1_levels = ['PhaseOut', 'Stop&Reduce', 'Stop&Maintain', 'StatusQuo']
att_2_levels = ['HighInvestment&Int', 'HighInvestment&Int&Consideration', 'LowInvestment&LowConsideration', 'LowInvestment']
att_3_levels = ['HealthEdu', 'EnergyAccess', 'LowPrices', 'Transfers', 'NothingSoc']
att_4_levels = ['CreateJobs', 'EarlyPension', 'JobGuarantee', 'Retrain', 'NothingEco']
att_5_levels = ['CivilNGO', 'EnergySector', 'LabourUnion', 'LocalGov', 'Media', 'Researchers', 'CentralGov']



#Remember to add att_6
att_levels = [att_5_levels, att_4_levels, att_3_levels, att_2_levels, att_1_levels]

att_colors = ['red', 'blue', 'green', 'orange', 'purple']  # Colors for each attribute group

fig = go.Figure()

total_levels = sum(len(levels) for levels in att_levels)

# Loop through each attribute group and add the data to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model.params[f'att_{5-i}_{level}'] for level in levels]
    att_standard_errors = [model.bse[f'att_{5-i}_{level}'] for level in levels]

    relative_differences = [coeff - att_coefficients[-1] for coeff in att_coefficients]

    fig.add_trace(go.Scatter(
        x=relative_differences,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color='darkgray', size=10),
        orientation='h',
        showlegend = False,
    ))

    fig.add_shape(
        type="rect",
        x0=-1.5,  # Set a fixed value for x0, which is left side of the plot
        x1=1.5,  # Set the width of the shape to 1000 (right side of the plot)
        y0=total_levels - sum(len(l) for l in att_levels[i:]),  # Set y0 to the starting level index
        y1=total_levels - sum(len(l) for l in att_levels[i:]) + len(levels) - 1,  # Set y1 to the ending level index
        fillcolor=att_colors[i],
        opacity=0.1,  # Set the opacity for a light transparent effect
        layer="below",  # Place the rectangle below the scatter plot markers
    )

# Add a vertical line at x=0 for reference
fig.add_shape(type="line", x0=0, x1=0, y0=att_5_levels[0], y1=att_1_levels[-1], line=dict(color="gray", width=1, dash='dash'))

# Update the layout of the error bar plot
fig.update_layout(
    title='Relative Rating Differences',
    xaxis_title='',
    yaxis_title='Attribute Levels',
    yaxis=dict(categoryorder='array', categoryarray=att_5_levels),  # Set the categoryorder for y-axis based on att_1_levels
    xaxis=dict(tickformat='.2f', zeroline=False),  # Remove x-axis zeroline
    showlegend=True,  # Show legend with attribute names
    margin=dict(l=80, r=30, b=40, t=80),
    height=600,  # Set the height of the plot to 600 pixels
    width=1000,
    title_x=0.62,
)

# Show the interactive error bar plot
fig.show()


Make it into MM Marginal Means!

In [35]:
import plotly.graph_objects as go

# Assuming 'model' is the variable that holds your regression results
# Extract the coefficients and standard errors for each attribute level
att_1_levels = ['PhaseOut', 'Stop&Reduce', 'Stop&Maintain', 'StatusQuo']
att_2_levels = ['HighInvestment&Int', 'HighInvestment&Int&Consideration', 'LowInvestment&LowConsideration', 'LowInvestment']
att_3_levels = ['HealthEdu', 'EnergyAccess', 'LowPrices', 'Transfers', 'NothingSoc']
att_4_levels = ['CreateJobs', 'EarlyPension', 'JobGuarantee', 'Retrain', 'NothingEco']
att_5_levels = ['CivilNGO', 'EnergySector', 'LabourUnion', 'LocalGov', 'Media', 'Researchers', 'CentralGov']



#Remember to add att_6
att_levels = [att_5_levels, att_4_levels, att_3_levels, att_2_levels, att_1_levels]

att_colors_control = ['red', 'blue', 'green', 'orange', 'purple']  # Colors for each attribute group

fig = go.Figure()

total_levels = sum(len(levels) for levels in att_levels)

# Loop through each attribute group and add the data to the plot
fig = go.Figure()

# Loop through each attribute group and add the data for 'control' to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model_control.params[f'att_{5-i}_{level}'] for level in levels]
    att_standard_errors = [model_control.bse[f'att_{5-i}_{level}'] for level in levels]

    relative_differences = [coeff - att_coefficients[-1] for coeff in att_coefficients]

    fig.add_trace(go.Scatter(
        x=relative_differences,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color='darkgray', size=10),
        orientation='h',
        showlegend=False,
        name='Control',  # Add a legend name for the control group
    ))

# Loop through each attribute group and add the data for 'treated' to the plot
for i, levels in enumerate(att_levels):
    att_coefficients = [model_treated.params[f'att_{5-i}_{level}'] for level in levels]
    att_standard_errors = [model_treated.bse[f'att_{5-i}_{level}'] for level in levels]

    relative_differences = [coeff - att_coefficients[-1] for coeff in att_coefficients]

    fig.add_trace(go.Scatter(
        x=relative_differences,
        y=levels,
        mode='markers',
        error_x=dict(type='data', array=att_standard_errors, color=att_colors[i], thickness=1.5),
        marker=dict(color=att_colors[i], size=10),  # Use different colors for treated group
        orientation='h',
        showlegend=False,
        name='Treated',  # Add a legend name for the treated group
    ))

    fig.add_shape(
        type="rect",
        x0=-1.5,  # Set a fixed value for x0, which is left side of the plot
        x1=1.5,  # Set the width of the shape to 1000 (right side of the plot)
        y0=total_levels - sum(len(l) for l in att_levels[i:]),  # Set y0 to the starting level index
        y1=total_levels - sum(len(l) for l in att_levels[i:]) + len(levels) - 1,  # Set y1 to the ending level index
        fillcolor=att_colors[i],
        opacity=0.1,  # Set the opacity for a light transparent effect
        layer="below",  # Place the rectangle below the scatter plot markers
    )

# Add a vertical line at x=0 for reference
fig.add_shape(type="line", x0=0, x1=0, y0=att_5_levels[0], y1=att_1_levels[-1], line=dict(color="gray", width=1, dash='dash'))

# Update the layout of the error bar plot
fig.update_layout(
    title='Relative Rating Differences',
    xaxis_title='',
    yaxis_title='Attribute Levels',
    yaxis=dict(categoryorder='array', categoryarray=att_5_levels),  # Set the categoryorder for y-axis based on att_1_levels
    xaxis=dict(tickformat='.2f', zeroline=False),  # Remove x-axis zeroline
    showlegend=True,  # Show legend with attribute names
    margin=dict(l=80, r=30, b=40, t=80),
    height=600,  # Set the height of the plot to 600 pixels
    width=1000,
    title_x=0.62,
)

# Show the interactive error bar plot
fig.show()


##### 1.2 Rating attributes with a Normalization Method.

*Normalizing the coefficients involves transforming them to a common scale, typically between 0 and 1. This makes it easier to compare the relative importance of different attributes.*


To aggregate the importance scores for different levels of the same attribute and obtain an overall importance score for each attribute, you can calculate a weighted average or sum of the importance scores of its individual levels. Here's how you can do it:

Calculate Normalized Importance Scores (NIS) for Attribute Levels:

Follow the normalization method as described earlier to calculate the normalized importance scores (NIS) for each attribute level.
Aggregate Importance Scores for Each Attribute:

a. Weighted Average Method:

Calculate the weighted average importance score for each attribute by taking the sum of the products of each level's NIS and its corresponding weight (frequency or probability of that level's occurrence in the choice sets).
This method considers both the relative importance of each level and its likelihood of being chosen in the experiment.
b. Simple Sum Method:

Sum up the normalized importance scores (NIS) of all levels within an attribute.
This method treats all levels equally in terms of their contribution to the overall importance score.
Attribute Importance Ranking:

Rank the attributes based on the aggregated importance scores. Higher scores indicate greater importance.
Example using the Weighted Average Method:

Let's consider an example with attribute "att_1" from your regression results. You have four levels: "PhaseOut," "StatusQuo," "Stop&Maintain," and "Stop&Reduce." You've already calculated the normalized importance scores (NIS) for each level as follows:

NIS(PhaseOut) = 0.8810 / 0.8810 = 1.0000
NIS(StatusQuo) = 0.2979 / 0.8810 = 0.3379
NIS(Stop&Maintain) = 0.1471 / 0.8810 = 0.1668
NIS(Stop&Reduce) = 0.9077 / 0.8810 = 1.0302
Let's assume that the frequency (or probability) of each level's occurrence in the choice sets is as follows:

PhaseOut: 30%
StatusQuo: 20%
Stop&Maintain: 25%
Stop&Reduce: 25%
Now, calculate the weighted average importance score for "att_1":
Weighted Average Importance Score for att_1 = (NIS(PhaseOut) * 0.30) + (NIS(StatusQuo) * 0.20) + (NIS(Stop&Maintain) * 0.25) + (NIS(Stop&Reduce) * 0.25)
Weighted Average Importance Score for att_1 = (1.0000 * 0.30) + (0.3379 * 0.20) + (0.1668 * 0.25) + (1.0302 * 0.25) ≈ 0.5623

Repeat this process for each attribute to obtain aggregated importance scores, and then rank the attributes based on these scores.

Remember that the choice of the weighting scheme (equal weights, frequency-based weights, or other relevant weights) depends on your specific context and research design.

##### Plot freq

In [25]:
freq = pd.read_csv(OUT / 'data' / 'data_freq.csv')

NIS

##### 1.3 General support plot for phase out

##### 1.4 Maybe: PCA Analysis:

Data Preparation:

Prepare your DCE data matrix, where each row represents a respondent's choice set, and columns represent different attribute levels.
Standardization:

Standardize the data by subtracting the mean and dividing by the standard deviation for each attribute. This ensures that all attributes are on similar scales and prevents attributes with larger variances from dominating the PCA.
Perform PCA:

Apply PCA to the standardized data matrix. The output of PCA will include the principal components and their associated eigenvalues.
Interpretation:

Examine the explained variance for each principal component. This helps you understand how much of the total variance in the data each component explains.
Look at the loadings (weights) of the original attributes on each principal component. These loadings indicate the strength and direction of the relationship between the attribute and the principal component.
Attribute Relationships:

PCA can provide insights into how attributes are related to each other. For example, attributes that have high loadings on the same principal component are positively correlated, while those with opposite loadings are negatively correlated.
Decision Support:

While PCA itself may not directly provide attribute importance scores for policy package choices, the derived principal components can help you identify patterns or relationships that might influence choices. These insights can then be used in conjunction with other methods to understand attribute importance.